## 1. Cargamos los datos a Preprocesar y limpiar (desde el BlobStorage)

In [0]:
#Librerias necesarias para usar el Azure DataLake y DataBrinks
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import io

# 2. Obtener conection Azure DataLake,interfaz de Azure(Claves de acceso: Key1) (Debes comentar esta variable Si NO deja hacer COMMIT DEL 
# CODIGO EN GIT,GITHUB)
# connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakemlopsd4m;AccountKey=iWT8t74/#XlqcqoR03keDVtFZPzr0PB9zDffMPaLWMUBIAjUww8uYAVkc9xRkcBtvTmUHKBvd1sB3+ASt6mGgcQ==;EndpointSuffix=core.windows.net'

# 3. Conectar al Blob Storage de Azure
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# 4. Identificamos el nombre del contenedor(container_name) y nombre del archivo(definir blob_name) en el Blob Storage
container_name = "raw/proyectocongestion_raw/fuentedatos_consolidado/"
blob_name = "datos_raw_consolidado.csv"

# 5. Obtener el blob_client
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# 6. Leer el contenido del blob como texto
blob_data = blob_client.download_blob().content_as_text()

# 7. Leer el archivo CSV en un DataFrame de Pandas desde el texto
datos = pd.read_csv(io.StringIO(blob_data))
datos.head()

In [0]:
datos.shape

In [0]:
datos["eq_id"].unique()